In [1]:
import pandas as pd
import glob
import re
import os

In [2]:
# Folder containing JPM Excel files
DATA_DIR = "./raw_data/"

# Grab all JPM_YYYYQQ.xlsx files
files = sorted(glob.glob(os.path.join(DATA_DIR, "JPM_*.xls")))

print(f"Found {len(files)} files")

Found 35 files


In [3]:
def extract_year_quarter(filename):
    match = re.search(r"(\d{4})Q([1-4])", filename)
    if not match:
        raise ValueError(f"Cannot parse year/quarter from {filename}")

    year = int(match.group(1))
    quarter = int(match.group(2))

    return year, quarter

In [27]:
all_data = []

In [28]:
# def get_sheet_name(year, quarter):
#     # 2010Q3 is the cutoff
#     # if year in (2011, 2012, 2013):
#     #     return "consolidated financial hig"
#     if (year < 2010) or (year == 2010 and quarter < 3):
#         return "jpmorgan chase  co consoli-1"
#     else:
#         return "consolidated financial hig"

def get_skip_rows(year, quarter):
    # if year in (2011, 2012, 2013):
    #     return 6
    if (year == 2020 and quarter < 3):
        return 6
    else:
        return 5

In [29]:
metrics_to_keep = [
    "Net charge-off rate",
    "Consumer net charge-off rate",
    "Net charge-offs",
    "Nonperforming assets"
]

In [30]:
# df = pd.read_excel(
#             "./raw_data/JPM_2010Q1.xls",
#             sheet_name="consolidated financial hig",
#             skiprows=6  # adjust if needed after checking
#         )
# df.rename(columns={df.columns[0]: "Metric"}, inplace=True)
# # Add time identifiers
# df["Year"] = 2010
# df["Quarter"] = 1
# df["Source"] = "JPM 10-Q"

# df_filtered = df[df["Metric"].isin(metrics_to_keep)]
# # df_long = df_filtered.melt(id_vars="Metric",var_name="Quarter",value_name="Value")
# # df_filtered = df_filtered[df_filtered['Quarter'].str[:7] != "Unnamed"]
# df_filtered = df_filtered[df_filtered.columns.drop(list(df.filter(regex='Unnamed:')))]
# df_filtered

In [31]:
for file in files:
    year, quarter = extract_year_quarter(file)

    try:
        df = pd.read_excel(
            file,
            sheet_name="consolidated financial hig",
            skiprows=get_skip_rows(year, quarter)  # adjust if needed after checking
        )

        # # Keep only relevant columns
        # df = df[COLUMNS_TO_KEEP].copy()

        df.rename(columns={df.columns[0]: "Metric"}, inplace=True)

        # Add time identifiers
        # df["Year"] = year
        # df["Quarter"] = quarter
        # df["Source"] = "JPM 10-Q"

        df_filtered = df[df["Metric"].isin(metrics_to_keep)]
        df_long = df_filtered.melt(id_vars="Metric",var_name="Quarter",value_name="Value")
        df_long = df_long[df_long['Quarter'].str[:7] != "Unnamed"]
        # df_filtered = df_filtered[df_filtered.columns.drop(list(df.filter(regex='Unnamed:')))]

        all_data.append(df_long)
        # merged_df = pd.merge(df_filtered_1, df_filtered, on='Metric', how='left')

    except Exception as e:
        print(f"Error processing {file}: {e}")

In [33]:
jpm_df = pd.concat(all_data, ignore_index=True)
jpm_df = jpm_df[jpm_df['Quarter'].str[1] == "Q"]
jpm_df
# merged_df

,Metric,Quarter,Value
38,Nonperforming assets,1Q18,6364
39,Nonperforming assets,4Q17,6426
40,Nonperforming assets,3Q17,6154
41,Nonperforming assets,2Q17,6432
42,Nonperforming assets,1Q17,6826
...,...,...,...
169,Net charge-offs,3Q21,524
170,Net charge-off rate,3Q21,0.21%
171,Nonperforming assets,2Q21,9802
172,Net charge-offs,2Q21,734


In [10]:
# jpm_df["Quarter_Num"] = jpm_df["Quarter"].str.replace("Q", "").astype(int)

# # Create a proper date column
# jpm_df["Date"] = pd.PeriodIndex(
#     year=jpm_df["Year"],
#     quarter=jpm_df["Quarter_Num"],
#     freq="Q"
# ).to_timestamp()

In [ ]:
OUTPUT_PATH = "/output/jpm_retail_portfolio_raw.csv"

jpm_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved cleaned JPM data to {OUTPUT_PATH}")